# Encontro 05: Simulação de Centralidades

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'

O objetivo desta atividade é realizar $24$ simulações de centralidade diferentes, para avaliar o desempenho de medidas clássicas em relação a diferentes processos de fluxo. Dessas $24$ simulações, são $12$ sobre o grafo `g1` e $12$ sobre o grafo `g2`.

In [3]:
g1 = sn.load_graph('renaissance.gml', has_pos=True)
g2 = sn.load_graph('../encontro02/1-introducao.gml', has_pos=True)

O primeiro grafo corresponde aos casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett, C. K. Ansell, 1993. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319.

In [4]:
sn.show_graph(g1, nlab=True)

O segundo grafo corresponde ao estudo de caso do primeiro encontro.

In [5]:
sn.show_graph(g2, nlab=True)

Das $12$ simulações sobre um dos grafos, são $6$ de *closeness* e $6$ de *betweenness*:

* duplicação serial através de caminhos;
* transferência através de caminhos;
* duplicação serial através de trilhas;
* transferência através de trilhas;
* duplicação serial através de passeios;
* transferência através de passeios.

In [6]:
from random import choice

TIMES = 1000

Em uma simulação de *closeness*, para cada origem `s` e destino `t`, mede-se o tempo que o fluxo leva para chegar de `s` a `t`. O *closeness simulado* de um nó `s` é a soma de todos os tempos medidos quando esse nó é uma origem. Como o fluxo pode ter passos aleatórios, o processo é repetido `TIMES` vezes e considera-se a média.

A função abaixo calcula o closeness simulado em relação a *transferência através de geodésicas*.

In [7]:
def simulate_closeness_transfer_geodesic(g):

    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                    m = choice(g.node[n]['shortest_neighbors'])
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                    g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

Vamos comparar o closeness simulado com o *closeness teórico*.

Para calcular o segundo, basta usar o *algoritmo de busca em largura* que vimos em encontros anteriores.

In [16]:
from math import inf, isinf
from queue import Queue

def build_closeness(g):
    for s in g.nodes():
        # início da busca em largura

        q = Queue()

        for n in g.nodes():
            g.node[n]['d'] = inf

        g.node[s]['d'] = 0
        q.put(s)

        while not q.empty():
            n = q.get()

            for m in g.neighbors(n):
                if isinf(g.node[m]['d']):
                    g.node[m]['d'] = g.node[n]['d'] + 1
                    q.put(m)

        # fim da busca em largura

        g.node[s]['theoretical_closeness'] = 0

        for n in g.nodes():
            g.node[s]['theoretical_closeness'] += g.node[n]['d']

Comparação de closeness no grafo `g1`:

*(vai demorar alguns segundos)*

In [ ]:
build_closeness(g1)

simulate_closeness_transfer_geodesic(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_closeness'], g1.node[n]['simulated_closeness'])

Comparação de closeness no grafo `g2`:

*(vai demorar alguns segundos)*

In [20]:
build_closeness(g2)

simulate_closeness_transfer_geodesic(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_closeness'], g2.node[n]['simulated_closeness'])

Rogerio 30 30.0
Roberto 36 36.0
Renato 47 47.0
Larissa 36 36.0
Jorge 49 49.0
Sueli 45 45.0
Conrado 30 30.0
Ricardo 38 38.0
Pamela 47 47.0
Fabio 49 49.0
Paulo 35 35.0
William 31 31.0
Tiago 38 38.0
Sandra 44 44.0
Patrick 41 41.0
Jose 40 40.0
Caio 46 46.0


Em uma simulação de *betweenness*, para cada origem `s`, destino `t` e intermediário `n`, mede-se a quantidade de vezes que o fluxo passa por `n` antes de chegar de `s` a `t`. O *betweenness simulado* de um nó `n` é a soma de todas as quantidades medidas quando esse nó é um intermediário. Como o fluxo pode ter passos aleatórios, o processo é repetido `TIMES` vezes e considera-se a média.

A função abaixo calcula o betweenness simulado em relação a *transferência através de geodésicas*.

In [7]:
def simulate_betweenness_transfer_geodesic(g):

    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:

                                    # TRANSFERÊNCIA: Escolhemos aleatoriamente um dos vizinhos válidos.
                                    # GEODÉSICA: Os vizinhos válidos são os que pertencem a geodésicas.
                                    m = choice(g.node[n]['shortest_neighbors'])
                                    nodes_reached.add(m)

                                    # TRANSFERÊNCIA: O fluxo transfere o bem para os nós que o fluxo
                                    # alcança, portanto o nó deixa de ser dono. Nos processos baseados
                                    # em duplicação, a linha abaixo não pode ser executada.
                                    g.node[n]['owner'] = False

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo e tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_abetweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

Vamos comparar o betweenness simulado com o *betweennesss teórico*.

Para calcular o segundo, basta usar a função caixa-preta `build_betweenness`. Vocês vão aprender a abrir essa caixa-preta em encontros posteriores.

Comparação de betweenness no grafo `g1`:

*(vai demorar alguns segundos)*

In [10]:
sn.build_betweenness(g1)

simulate_betweenness_transfer_geodesic(g1)

for n in g1.nodes():
    print(g1.node[n]['label'], g1.node[n]['theoretical_betweenness'], g1.node[n]['simulated_betweenness'])

NameError: name 'simulate_betweenness_transfer_geodesic' is not defined

Comparação de betweenness no grafo `g2`:

*(vai demorar alguns segundos)*

In [ ]:
sn.build_betweenness(g2)

simulate_betweenness_transfer_geodesic(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['theoretical_betweenness'], g2.node[n]['simulated_betweenness'])

## Entregáveis

Para **quinta 24/8**, você deve entregar todas as funções abaixo.

Funções auxiliares para evitar repetição de código são permitidas e encorajadas.

In [7]:
def simulate_closeness_serial_path(g):
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    for i in g.neighbors(n):
                                        if not g.node[i]['owner']:
                                            available_neighbors.append(i)
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)



                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

In [8]:
simulate_closeness_serial_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_closeness'])

Rogerio 69.942
Roberto 73.298
Renato 82.831
Larissa 75.786
Jorge 80.414
Sueli 80.296
Conrado 60.814
Ricardo 75.144
Pamela 80.502
Fabio 80.41
Paulo 75.133
William 68.924
Tiago 75.561
Sandra 81.078
Patrick 71.737
Jose 75.875
Caio 82.902


In [9]:
def simulate_closeness_transfer_path(g):
        # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    for i in g.neighbors(n):
                                        if not g.node[i]['owner']:
                                            available_neighbors.append(i)
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)
                                        g.node[n]['owner'] = False



                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES
    pass

In [10]:
simulate_closeness_transfer_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_closeness'])

Rogerio 679.234
Roberto 646.243
Renato 625.218
Larissa 675.67
Jorge 661.316
Sueli 630.363
Conrado 653.976
Ricardo 660.356
Pamela 625.97
Fabio 663.866
Paulo 641.697
William 680.419
Tiago 681.773
Sandra 630.932
Patrick 674.606
Jose 676.808
Caio 626.647


In [13]:
def simulate_closeness_serial_trail(g):
   # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    available_edges = []
                                    for i in g.neighbors(n):
                                        if [i,n] not in available_edges and [n,i] not in available_edges:
                                            available_neighbors.append(i)
                                            available_edges.append([i,n])
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)



                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

In [14]:
simulate_closeness_serial_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_closeness'])

Rogerio 104.56
Roberto 108.633
Renato 128.869
Larissa 116.616
Jorge 125.51
Sueli 124.528
Conrado 82.69
Ricardo 115.867
Pamela 125.86
Fabio 125.718
Paulo 112.067
William 102.027
Tiago 115.996
Sandra 125.43
Patrick 110.883
Jose 116.849
Caio 129.337


In [15]:
def simulate_closeness_transfer_trail(g):
       # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    available_edges = []
                                    for i in g.neighbors(n):
                                        if [i,n] not in available_edges and [n,i] not in available_edges:
                                            available_neighbors.append(i)
                                            available_edges.append([i,n])
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)
                                        g.node[n]['owner'] = False



                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

In [16]:
simulate_closeness_transfer_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_closeness'])

Rogerio 682.496
Roberto 646.844
Renato 634.651
Larissa 675.846
Jorge 664.73
Sueli 631.726
Conrado 656.652
Ricardo 671.018
Pamela 626.268
Fabio 652.865
Paulo 643.056
William 671.474
Tiago 679.093
Sandra 632.59
Patrick 660.673
Jose 684.244
Caio 630.896


In [17]:
def simulate_closeness_serial_walk(g):
       # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    m = choice(g.neighbors(n))
                                    nodes_reached.add(m)


                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

In [18]:
simulate_closeness_serial_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_closeness'])

Rogerio 103.742
Roberto 108.699
Renato 129.752
Larissa 116.262
Jorge 125.761
Sueli 124.833
Conrado 82.663
Ricardo 115.143
Pamela 125.802
Fabio 126.045
Paulo 111.916
William 102.11
Tiago 116.365
Sandra 125.201
Patrick 110.996
Jose 117.547
Caio 128.326


In [19]:
def simulate_closeness_transfer_walk(g):
           # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_closeness'] = 0

    for _ in range(TIMES):
        for s in g.nodes():

            # Inicialização do closeness de s.
            g.node[s]['closeness'] = 0

            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        time = 1

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    m = choice(g.neighbors(n))
                                    nodes_reached.add(m)
                                    g.node[n]['owner'] = False

                                    


                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            time += 1

                    # Soma do tempo de s a t ao closeness de s.
                    g.node[s]['closeness'] += time

        # Incremento das médias.
        for n in g.nodes():
            g.node[n]['simulated_closeness'] += g.node[n]['closeness']

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_closeness'] /= TIMES

In [20]:
simulate_closeness_transfer_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_closeness'])

Rogerio 683.967
Roberto 636.423
Renato 628.27
Larissa 674.19
Jorge 661.669
Sueli 644.272
Conrado 661.159
Ricardo 671.815
Pamela 624.693
Fabio 674.587
Paulo 640.62
William 682.245
Tiago 675.596
Sandra 631.224
Patrick 668.061
Jose 685.148
Caio 628.678


In [21]:
def simulate_betweenness_serial_path(g):
    
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    for i in g.neighbors(n):
                                        if not g.node[i]['owner']:
                                            available_neighbors.append(i)
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [23]:
simulate_betweenness_serial_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_betweenness'])

Rogerio 76.3335
Roberto 64.6735
Renato 37.3295
Larissa 64.838
Jorge 38.0505
Sueli 48.198
Conrado 73.2925
Ricardo 59.0555
Pamela 39.506
Fabio 37.933
Paulo 66.7225
William 73.981
Tiago 58.918
Sandra 50.416
Patrick 47.755
Jose 47.607
Caio 41.0375


In [24]:
def simulate_betweenness_transfer_path(g):
        
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    for i in g.neighbors(n):
                                        if not g.node[i]['owner']:
                                            available_neighbors.append(i)
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)
                                        g.node[n]['owner'] = False


                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [25]:
simulate_betweenness_transfer_path(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_betweenness'])

Rogerio 76.3335
Roberto 64.6735
Renato 37.3295
Larissa 64.838
Jorge 38.0505
Sueli 48.198
Conrado 73.2925
Ricardo 59.0555
Pamela 39.506
Fabio 37.933
Paulo 66.7225
William 73.981
Tiago 58.918
Sandra 50.416
Patrick 47.755
Jose 47.607
Caio 41.0375


In [26]:
def simulate_betweenness_serial_trail(g):
           
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    available_edges = []
                                    for i in g.neighbors(n):
                                        if [i,n] not in available_edges and [n,i] not in available_edges:
                                            available_neighbors.append(i)
                                            available_edges.append([i,n])
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)

                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [27]:
simulate_betweenness_serial_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_betweenness'])

Rogerio 276.426
Roberto 217.143
Renato 120.2505
Larissa 282.2615
Jorge 126.5015
Sueli 195.283
Conrado 185.4945
Ricardo 225.7365
Pamela 121.363
Fabio 126.477
Paulo 244.3
William 248.3355
Tiago 226.0175
Sandra 224.485
Patrick 119.339
Jose 146.716
Caio 158.895


In [28]:
def simulate_betweenness_transfer_trail(g):
              
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                if g.node[n]['owner']:
                                    available_neighbors = []
                                    available_edges = []
                                    for i in g.neighbors(n):
                                        if [i,n] not in available_edges and [n,i] not in available_edges:
                                            available_neighbors.append(i)
                                            available_edges.append([i,n])
                                    if  available_neighbors:
                                        m = choice(available_neighbors)
                                        nodes_reached.add(m)
                                        g.node[n]['owner'] = False



                            # Todos os nós qu
                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [29]:
simulate_betweenness_transfer_trail(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_betweenness'])

Rogerio 429.2205
Roberto 301.4025
Renato 174.92
Larissa 487.195
Jorge 175.9555
Sueli 298.397
Conrado 246.9145
Ricardo 363.67
Pamela 175.3055
Fabio 176.74
Paulo 362.6425
William 366.1105
Tiago 363.7075
Sandra 358.1515
Patrick 178.261
Jose 240.0005
Caio 235.993


In [30]:
def simulate_betweenness_serial_walk(g):
                 
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                 if g.node[n]['owner']:
                                    m = choice(g.neighbors(n))
                                    nodes_reached.add(m)



                            # Todos os nós qu
                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [ ]:
simulate_betweenness_serial_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_betweenness'])

In [31]:
def simulate_betweenness_transfer_walk(g):
                    
    # Inicialização das médias.

    for n in g.nodes():
        g.node[n]['simulated_betweenness'] = 0

    for _ in range(TIMES):

        # Inicialização de todos os betweenness.
        for n in g.nodes():
            g.node[n]['betweenness'] = 0

        for s in g.nodes():
            for t in g.nodes():
                if s != t:

                    # Função caixa-preta que calcula, para cada nó, seu subconjunto
                    # de vizinhos que pertencem a geodésicas de s a t. Esse subconjunto
                    # é armazenado no atributo shortest_neighbors. Vocês vão aprender
                    # a abrir essa caixa-preta em encontros posteriores.
                    sn.build_shortest_paths(g, s, t)

                    # Dependendo do processo, o fluxo pode não ter sucesso, ou seja,
                    # pode ficar preso em uma parte do grafo sem nunca atingir t.
                    # Quando isso acontece, simplesmente tenta-se novamente.

                    success = False

                    while not success:

                        # Chamamos de "dono" um nó que possui o bem conduzido pelo
                        # fluxo. No início do processo, sabemos que o único dono é s.
                        for n in g.nodes():
                            g.node[n]['owner'] = False
                        g.node[s]['owner'] = True

                        for n in g.nodes():
                            if n != s and n != t:
                                g.node[n]['partial_betweenness'] = 0

                        while True:

                            # O conjunto nodes_reached indica os nós que o fluxo
                            # alcança ao "avançar mais um passo".
                            nodes_reached = set()

                            for n in g.nodes():
                                 if g.node[n]['owner']:
                                    m = choice(g.neighbors(n))
                                    nodes_reached.add(m)
                                    g.node[n]['owner'] = False


                            # Todos os nós qu
                            # Todos os nós que o fluxo alcança tornam-se donos.
                            for n in nodes_reached:
                                g.node[n]['owner'] = True

                            # Se alcançamos t, interrompemos o fluxo e paramos de tentar.
                            if t in nodes_reached:
                                success = True
                                break

                            # Se não alcançamos ninguém, interrompemos o fluxo tentamos novamente.
                            if not nodes_reached:
                                break

                            for n in nodes_reached:
                                if n != s and n != t:
                                    g.node[n]['partial_betweenness'] += 1

                    # Soma de todos os betweenness parciais dos intermediários.
                    for n in g.nodes():
                        if n != s and n != t:
                            g.node[n]['betweenness'] += g.node[n]['partial_betweenness']

        # Incremento das médias. Divide-se o valor por 2 para
        # desconsiderar a simetria de um grafo não-dirigido.
        for n in g.nodes():
            g.node[n]['simulated_betweenness'] += g.node[n]['betweenness'] / 2

    # Finalização das médias.
    for n in g.nodes():
        g.node[n]['simulated_betweenness'] /= TIMES

In [32]:
simulate_betweenness_transfer_walk(g2)

for n in g2.nodes():
    print(g2.node[n]['label'], g2.node[n]['simulated_betweenness'])

Rogerio 431.0265
Roberto 301.655
Renato 174.4935
Larissa 491.1315
Jorge 178.021
Sueli 296.9675
Conrado 247.762
Ricardo 366.54
Pamela 175.435
Fabio 177.858
Paulo 362.322
William 369.95
Tiago 365.393
Sandra 358.3905
Patrick 179.6575
Jose 240.6715
Caio 235.547


Para encontros posteriores, serão pedidas análises dos resultados dessas simulações.


## Avançado

Na consolidação do módulo de centralidade, você pode atingir conceito avançado no objetivo **traduzir conceitos sociológicos para teoria dos grafos e algoritmos em grafos** se formular e implementar uma proposta coerente de *degree simulado*. Essa proposta deve ser entregue na mesma data.